## Principal Component Analysis

In this notebook, PCA will be implemented both from scratch, and from sklearn.

In [2]:
import pandas as pd
import numpy as np
from numpy.linalg import eig
from sklearn.decomposition import PCA

In [11]:
# create data
df = np.array([[3, 4], [2, 8], [6, 9]])
print(df)

[[3 4]
 [2 8]
 [6 9]]


In [12]:
# make data frame
df = pd.DataFrame(df, columns = ['Math', 'Science'])
df

,Math,Science
0,3,4
1,2,8
2,6,9


#### Center the data by mean

In [13]:
# center the data frame
df_centered = df.apply(lambda x: x - x.mean())
df_centered

,Math,Science
0,-0.666667,-3.0
1,-1.666667,1.0
2,2.333333,2.0


#### Get Covariance Matrix

To find the covariance matrix, we have to use the transpose of our scaled data because we want to work row-wise.

In [21]:
# covariance matrix
cov_matrix = np.cov(df_centered.T)
cov_matrix

array([[4.33333333, 2.5       ],
       [2.5       , 7.        ]])

#### Find the Eigenvalue and Eigenvectors for covariance matrix.

This code finds the eigenvalues and the eigenvectors. We get two eigenvalues and two eigenvectors for our data.

In [22]:
# find eigenvalue and eigenvector
eigval, eigvec = eig(cov_matrix)
eigval, eigvec

(array([2.83333333, 8.5       ]),
 array([[-0.85749293, -0.51449576],
        [ 0.51449576, -0.85749293]]))

#### Project our scaled data on to the new axes (eigvec)

In [23]:
# project scaled data on to eigenvectors (new axes)
projected = eigvec.T.dot(df_centered.T)
projected.T

array([[-9.71825316e-01,  2.91547595e+00],
       [ 1.94365063e+00,  1.11022302e-16],
       [-9.71825316e-01, -2.91547595e+00]])

In [25]:
# make them a dataframe
pca_df = pd.DataFrame(projected.T, columns = ['pc1', 'pc2'])
pca_df

,pc1,pc2
0,-0.971825,2.915476e+00
1,1.943651,1.110223e-16
2,-0.971825,-2.915476e+00


#### Using Sklearn

In this section, we implement the principal component analysis using the Scikit-Learn library. This library implements the centering of the data for us so we do not have to worry about that part. We see from the results that the same principal components are found with axes flipped.

In [35]:
# use sklearn
pca = PCA(n_components = 2)
pca_df_sklearn = pca.fit_transform(df)
pca_df_sklearn = pd.DataFrame(pca_df_sklearn, columns = ['pc1', 'pc2'])
pca_df_sklearn

,pc1,pc2
0,-2.915476e+00,0.971825
1,7.375885e-16,-1.943651
2,2.915476e+00,0.971825


In [34]:
# see if our eigenvectors are the same as those produced by sklearn
print(f'{pca.components_.T[:, 0:2]}\n\n{eigvec}')

[[ 0.51449576  0.85749293]
 [ 0.85749293 -0.51449576]]

[[-0.85749293 -0.51449576]
 [ 0.51449576 -0.85749293]]


##### Explained Variance Ratio

Since we are only using a simple two variable dataset, the two principal components will contain all 100% of the explained variance.

In [36]:
pca.explained_variance_ratio_

array([0.75, 0.25])

##### 3-D Dataset

In [112]:
# create dataset
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [120]:
# center the data around the mean
x_centered = X -X.mean(axis=0)

# get eigenvectors using singular value decomposition with numpy
U, s, Vt = np.linalg.svd(x_centered)

# the transpose of Vt gives the principal components. 
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [121]:
# projecting on to the plane
# matrix containing the columns of V1
W2 = Vt.T[:, :2]
X2D = x_centered.dot(W2)

In [122]:
# using sklearn
pca = PCA(n_components= 2)
X2 = pca.fit_transform(X)



In [123]:
X2D[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

In [124]:
X2[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

Below we can see how to calculate the explained variance using both SVD and sklearn. The values are the same. For sklearn, we only get two numbers because we chose 2 principal components. When we did SVD we have three because we had three variables in our data. These add up to 100 percent of the total variance. 

In [125]:
# s is equal to the variance of each principal component. 
# to find explained var we square it and divide by degrees of freedom
explained_var = (s ** 2) / x_centered.shape[0]

# total var
total_var = np.sum(explained_var)

# ratio
explained_variance_ratio = explained_var / total_var

explained_variance_ratio


array([0.84248607, 0.14631839, 0.01119554])

In [126]:
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])